##### Copyright 2024 Kensho Technologies, LLC

# OpenAI Function Calling
**_GPT to retrieve data from the LLM-ready API using the kFinance python library!_**

What you'll need to run this notebook:

1.   kFinance credentials
2.   An OpenAI API key

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/kensho-technologies/llm-ready-api-examples/blob/main/function_calling/OpenAI_function_calling.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Install dependencies

In [ ]:
# install the latest version of kFinance package
%pip install https://kfinance.kensho.com/static/kensho_finance.tar.gz
# install the LLM Python package
%pip install openai

# Instantiate kFinance Client

In [ ]:
# import the kfinance client
from kensho_finance.kfinance import Client
# import standard libraries
import functools
import types
import json
import sys
# check if the current environment is a Google Colab
try:
  import google.colab
  IN_GOOGLE_COLAB = True
except:
  IN_GOOGLE_COLAB = False

# initialize the kfinance client with one of the following:
# 1. your kensho refresh token
# 2. your kensho client id and kensho private key
# 3. automated login (not accessible on Google Collab)
if IN_GOOGLE_COLAB:
    kensho_refresh_token = ""
    assert kensho_refresh_token != "", "kensho refresh token is empty! Make sure to enter your kensho refresh token above"
    kfinance_client = Client(refresh_token=kensho_refresh_token)

    # kensho_client_id = ""
    # kensho_private_key = ""
    # assert kensho_client_id != "", "kensho client id is empty! Make sure to enter your kensho client id above"
    # assert kensho_private_key != "", "kensho private key is empty! Make sure to enter your kensho private key above"
    # kfinance_client = Client(client_id=kensho_client_id, private_key=kensho_private_key)
else:
    kfinance_client = Client()

# the prompt given to all LLMs to instruct them how to use tools to call the kFinance API
SYSTEM_PROMPT = "You are an LLM trying to help financial analysts. Use the supplied tools to assist the user. Always use the `get_latest` function when asked about the last or most recent quarter or time period etc. Always use the `get_latest` function when a tool requires a time parameter and the time is unspecified in the question"
# the message shown to users to prompt them to input a message
USER_INPUT_PROMPT = "Enter your message and press the [return] key\n"

# OpenAI Function Calling

In [ ]:
# import OpenAI
from openai import OpenAI
# create a struct similar to the OpenAI response class, for manually creating responses
from collections import namedtuple
Message = namedtuple('Message', 'role content tool_calls')

# the OpenAIChat class is used to create a chat loop that automatically executes tool calls
class OpenAIChat:
  def __init__(self, kfinance_client: Client):
    # initialize OpenAI with your OpenAI API key
    openai_api_key = "" # replace with your own key
    assert openai_api_key != "", "OpenAI API key is empty! Make sure to enter your OpenAI API key above"
    # good client config for OpenAI. There are other ways to access OpenAI
    self.openai = OpenAI(
      api_key=openai_api_key,
    )
    # initialize the kFinance client
    self.kfinance_client = kfinance_client
    # initialize tools and tool descriptions
    self.tools = kfinance_client.tools
    self.tool_descriptions = kfinance_client.openai_tool_descriptions
    # set the first message to include the system prompt
    self.messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
    ]

  def print_response(self):
      """Print and return response"""
      # try to send the message history to OpenAI and get the response
      try:
          response = self.openai.chat.completions.create(
              model="gpt-4o", # you can use any OpenAI model that supports function calling
              messages=self.messages,
              tools=self.tool_descriptions,
          )
          response_message = response.choices[0].message
          self.messages.append(response_message)
      # if there's an error, manually create a response with the error
      except Exception as e:
          self.messages.append({"role": "user", "content": f"you had an error: {str(e)}"})
          response_message = Message("assistant", f"an error occured: {str(e)}", None)
      # if the response contains text (and not just tool calls), print out the text
      if response_message.content is not None:
            sys.stdout.write(response_message.content)
      return response_message

  def print_responses(self, user_input: str) -> list[str]:
      """Print responses and call tools"""
      # append the current user input as a message to the message history
      self.messages.append({"role": "user", "content": user_input})
      # get the OpenAI response to the message history
      response_message = self.print_response()
      # while the response has tool calls
      while response_message.tool_calls is not None:
          # for each tool call, execute the function and arguments specified in the tool call
          # and append the output as a message to the message history
          for tool_call in response_message.tool_calls:
              function = tool_call.function.name
              arguments = json.loads(tool_call.function.arguments)
              # try to execute the function and arguments specified in the tool call
              try:
                  output = self.tools[function](**arguments)
                  # append the output as a message to the message history
                  self.messages.append(
                      {
                          "role": "tool",
                          "content": json.dumps({"output": str(output)}),
                          "tool_call_id": tool_call.id,
                      }
                  )
              # if there's an exception thrown while executing the function,
              # append the exception as a message to the message history
              except Exception as e:
                  self.messages.append(
                      {
                          "role": "tool",
                          "content": json.dumps({"output": str(e)}),
                          "tool_call_id": tool_call.id,
                      }
                  )
          # get a new response
          response_message = self.print_response()
      return None

  def start_chatting(self) -> None:
      """Open chat shell"""
      # prompt for user input and get the OpenAI response in a loop
      while True:
          user_input = input(USER_INPUT_PROMPT)
          self.print_responses(user_input)
          sys.stdout.write("\n")

In [ ]:
# instantiate the OpenAIChat with the kfinance client
openai_chat = OpenAIChat(kfinance_client)
# start chatting with the OpenAIChat
openai_chat.start_chatting()